In [ ]:
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.1 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=5723ade3e567e2a3edd7cd745eb1de1ea5768b2a9e03c4c58ec5ca07405d921f
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [ ]:
!pip install gym pyvirtualdisplay

In [ ]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
# Create the CartPole environment
env = gym.make('CartPole-v1')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Initial Testing with Random Actions

In [ ]:
# Number of episodes
episodes = 5

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        new_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:20.0
Episode:2 Score:34.0
Episode:3 Score:17.0
Episode:4 Score:49.0
Episode:5 Score:11.0


In [ ]:
env.close()

Training Section

In [ ]:
#PPO Algorithm

# Create a single Gym environment
env_name = 'CartPole-v1'
env = gym.make(env_name)

# Wrap the environment with the custom wrapper and DummyVecEnv
# Parallel environments
vec_env = make_vec_env(env_name, n_envs=4)

# Create the PPO model
model = PPO('MlpPolicy', vec_env, verbose=1)

Using cuda device


In [ ]:
# Train the model
model.learn(total_timesteps=10000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23       |
|    ep_rew_mean     | 23       |
| time/              |          |
|    fps             | 2043     |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 8192     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 30.9         |
|    ep_rew_mean          | 30.9         |
| time/                   |              |
|    fps                  | 1250         |
|    iterations           | 2            |
|    time_elapsed         | 13           |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0152319055 |
|    clip_fraction        | 0.223        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.681       |
|    explained_variance   | -0.00249     |
|    learning_r

Evaluation Section

In [ ]:
#Evaluate the model
evaluate_policy(model, vec_env, n_eval_episodes=10, render=True)
#Prints(average reward per episode, standard deviation for reward)

(355.1, 109.792941485325)

In [ ]:
vec_env.close()

In [ ]:
env.close()

Testing Section

In [ ]:
# Number of episodes
episodes = 5

for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:20.0
Episode:2 Score:110.0
Episode:3 Score:35.0
Episode:4 Score:150.0
Episode:5 Score:33.0


In [ ]:
env.close()

Callback

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(vec_env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path='modelsavepath',
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', vec_env, verbose=1, tensorboard_log='logpath')

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

Changing Policies

In [ ]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [ ]:
model = PPO('MlpPolicy', vec_env, verbose=1, policy_kwargs={'net_arch':net_arch})

Using cpu device


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:484: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
vec_env.close()
env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Changing Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model= DQN('MlpPolicy', vec_env, verbose=1)

Using cpu device


In [ ]:
model.learn(total_timesteps=20000)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 18.8     |
|    ep_rew_mean      | 18.8     |
|    exploration_rate | 0.951    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 11158    |
|    time_elapsed     | 0        |
|    total_timesteps  | 104      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.9     |
|    ep_rew_mean      | 21.9     |
|    exploration_rate | 0.911    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 9796     |
|    time_elapsed     | 0        |
|    total_timesteps  | 188      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 20.8     |
|    ep_rew_mean      | 20.8     |
|    exploration_rate | 0.861    |
| time/               |          |
|    episodes       